<a href="https://colab.research.google.com/github/Kanisorn001/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [587]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic

In [588]:
# Load the Excel file
df_house = pd.read_excel('/content/Data House.xlsx')

In [589]:
df_house

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [590]:
df_house.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden'],
      dtype='object')

In [591]:
df_house.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1656 non-null   object 
 1   name_th             1655 non-null   object 
 2   name_en             1651 non-null   object 
 3   project_name        539 non-null    object 
 4   company             348 non-null    object 
 5   province            1656 non-null   object 
 6   district            1656 non-null   object 
 7   sub_district        1516 non-null   object 
 8   property_type       1656 non-null   object 
 9   home improvement    1457 non-null   object 
 10  renovate            1445 non-null   object 
 11  public utility      18 non-null     object 
 12  land type           13 non-null     object 
 13  land wide(m)        56 non-null     float64
 14  land long(m)        47 non-null     float64
 15  topsoil             19 non-null     object 
 16  corner

##เติม missing

In [592]:
selected_columns = ['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type','renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom',
                    'number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'zone',]
df_house = df_house[selected_columns]

In [593]:
# prompt: เติม missing ของ 'zone' จากข้อมูลที่มีอยู่

# Fill missing 'zone' values with the mode of 'zone'
df_house['zone'] = df_house['zone'].fillna(df_house['zone'].mode()[0])
df_house['district'] = df_house['district'].fillna(df_house['district'].mode()[0])
df_house['sub_district'] = df_house['sub_district'].fillna(df_house['sub_district'].mode()[0])

<ipython-input-593-4bd8d7e728af>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house['zone'] = df_house['zone'].fillna(df_house['zone'].mode()[0])
<ipython-input-593-4bd8d7e728af>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house['district'] = df_house['district'].fillna(df_house['district'].mode()[0])
<ipython-input-593-4bd8d7e728af>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [594]:
# Handle missing values (example: fill with mean)
for col in ['number_of_rai', 'number_of_ngan','number_of_va', 'usable_area']:
    df_house[col] = df_house[col].fillna(df_house[col].mean())


<ipython-input-594-cec4bfa9dfc5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house[col] = df_house[col].fillna(df_house[col].mean())


In [595]:
# Outlier Removal (example: IQR method)
# Define a function to remove outliers
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

In [596]:
print(df_house.head())

   id                                   name_th  \
0   1                   บ้านเดี่ยว ชนบท ขอนแก่น   
1   4                 บ้านเดี่ยว น้ำพอง ขอนแก่น   
2  16              บ้านเดี่ยว มัญจาคีรี ขอนแก่น   
3  18     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4  19  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   

                                    name_en    project_name  \
0                   บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                 บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
2              บ้านเดี่ยว มัญจาคีรี ขอนแก่น             NaN   
3     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   

                                 company province      district sub_district  \
0                                    NaN  ขอนแก่น          ชนบท  ศรีบุญเรือง   
1                                    NaN  ขอนแก่น        น้ำพอง      ทรายมูล   
2                                    NaN  ขอนแก่น     มัญจาคีรี      กุดเค

In [597]:
print(df_house.isnull().sum())

id                       0
name_th                  1
name_en                  5
project_name          1117
company               1308
province                 0
district                 0
sub_district             0
property_type            0
renovate               211
number_of_rai            0
number_of_ngan           0
number_of_va             0
number_of_story        549
number_of_bedroom       27
number_of_bathroom       0
direction                0
furniture                0
usable_area              0
number_of_parking      215
sell_price               1
zone                     0
dtype: int64


In [598]:
# prompt: เติม missing number_of_story number_of_bedroom number_of_parking   sell_price                      จากข้อมูลที่มีอยู่

# Fill missing values for 'number_of_story', 'number_of_bedroom', and 'number_of_parking'
# using the most frequent value (mode) for each column.
for col in ['renovate','number_of_story', 'number_of_bedroom', 'number_of_parking']:
    df_house[col] = df_house[col].fillna(df_house[col].mode()[0])

# Fill missing 'sell_price' values with the median. Using the median is generally better
# than the mean for skewed data like house prices.
df_house['sell_price'] = df_house['sell_price'].fillna(df_house['sell_price'].median())
print(df_house.isnull().sum())

id                       0
name_th                  1
name_en                  5
project_name          1117
company               1308
province                 0
district                 0
sub_district             0
property_type            0
renovate                 0
number_of_rai            0
number_of_ngan           0
number_of_va             0
number_of_story          0
number_of_bedroom        0
number_of_bathroom       0
direction                0
furniture                0
usable_area              0
number_of_parking        0
sell_price               0
zone                     0
dtype: int64


<ipython-input-598-33bfa49cd1ce>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house[col] = df_house[col].fillna(df_house[col].mode()[0])
<ipython-input-598-33bfa49cd1ce>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house['sell_price'] = df_house['sell_price'].fillna(df_house['sell_price'].median())


In [599]:
df_house

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,renovate,...,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,zone
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0,กลางเมือง-เมืองเก่า-กุดกว้าง
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0,กลางเมือง-เมืองเก่า-กุดกว้าง
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0,กลางเมือง-เมืองเก่า-กุดกว้าง
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0,มะลิวัลย์-ร.8-สนามบิน
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0,มะลิวัลย์-ร.8-สนามบิน
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,73.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,108.76,0.0,437400.0,กลางเมือง-เมืองเก่า-กุดกว้าง
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,84.0,1.0,2.0,1,ไม่ระบุ,ไม่ระบุ,119.00,0.0,714600.0,กลางเมือง-เมืองเก่า-กุดกว้าง
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,45.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,188.50,0.0,451000.0,กลางเมือง-เมืองเก่า-กุดกว้าง
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,59.0,2.0,1.0,1,ไม่ระบุ,ไม่ระบุ,207.00,0.0,910000.0,กลางเมือง-เมืองเก่า-กุดกว้าง


##JSON

In [600]:
with open('/content/school.json', 'r') as file:
    school_data = file.read(1000000)  # อ่านแค่ 1000000 ตัวอักษรแรก
    print(school_data)

{"1": [{"business_status": "OPERATIONAL", "geometry": {"location": {"lat": 16.1311875, "lng": 102.6504375}, "viewport": {"northeast": {"lat": 16.13271247989272, "lng": 102.6517733298927}, "southwest": {"lat": 16.13001282010728, "lng": 102.6490736701073}}}, "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/school-71.png", "icon_background_color": "#7B9EB0", "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/school_pinlet", "name": "Ban La Wa Health Center", "photos": [{"height": 3456, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/109948283369748538605\">A Google User</a>"], "photo_reference": "ARywPAKrl4GkxA4eldDQLd4bLsqag1nq-g5lacaHgemCiE3WPI4F53bqs0YxmhADKZMTh7Q_q-DsUUqNoYE6cCM8bVRfOXLaLqO2b6y97rIPMho8U7WwZPTg5LEdwjCRULM_5AkIgaxlYSSYfCgULGHDtro2GPqPC99E-JjwFCjz35fYkwqM", "width": 4608}], "place_id": "ChIJW5GGVOuCGDERn2dgufu1KOU", "rating": 4.5, "reference": "ChIJW5GGVOuCGDERn2dgufu1KOU", "scope": "GOOGLE", "types": ["

In [601]:

# Load 7-Eleven data
with open('/content/7-eleven.json', 'r') as f:
    seven_eleven_data = json.load(f)

def calculate_distances(house_lat, house_lon, data, radius):
    count = 0
    for item in data:
        try:
          place_lat = item['lat']
          place_lon = item['lng']
          distance = geodesic((house_lat, house_lon), (place_lat, place_lon)).km
          if distance <= radius:
              count += 1
        except:
          # Handle cases where latitude or longitude might be missing or invalid
          pass
    return count

# Assuming df_house is your DataFrame from the previous code
df_house['d1000-7/11'] = 0
df_house['d1000-school'] = 0

for index, row in df_house.iterrows():
    try:
        house_lat = row['lat']  # Replace with your actual latitude column name
        house_lon = row['lng'] # Replace with your actual longitude column name

        df_house.loc[index, 'd1000-7/11'] = calculate_distances(house_lat, house_lon, seven_eleven_data, 1)
        df_house.loc[index, 'd1000-school'] = calculate_distances(house_lat, house_lon, school_data, 1)
    except KeyError:
        print(f"Warning: Latitude or longitude not found for house ID {row['id']}. Skipping.")
    except Exception as e:
        print(f"An error occurred: {e}")

print(df_house.head())

<ipython-input-601-ab9e89945bad>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house['d1000-7/11'] = 0
<ipython-input-601-ab9e89945bad>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_house['d1000-school'] = 0


   id                                   name_th  \
0   1                   บ้านเดี่ยว ชนบท ขอนแก่น   
1   4                 บ้านเดี่ยว น้ำพอง ขอนแก่น   
2  16              บ้านเดี่ยว มัญจาคีรี ขอนแก่น   
3  18     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4  19  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   

                                    name_en    project_name  \
0                   บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                 บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
2              บ้านเดี่ยว มัญจาคีรี ขอนแก่น             NaN   
3     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   

                                 company province      district sub_district  \
0                                    NaN  ขอนแก่น          ชนบท  ศรีบุญเรือง   
1                                    NaN  ขอนแก่น        น้ำพอง      ทรายมูล   
2                                    NaN  ขอนแก่น     มัญจาคีรี      กุดเค

In [602]:
df_house

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,renovate,...,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,zone,d1000-7/11,d1000-school
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,2.0,2,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,1.0,1,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,11.0,11,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,3.0,3,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0,มะลิวัลย์-ร.8-สนามบิน,0,0
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,3.0,2,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0,มะลิวัลย์-ร.8-สนามบิน,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,2.0,1,ไม่ระบุ,ไม่ระบุ,108.76,0.0,437400.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,2.0,1,ไม่ระบุ,ไม่ระบุ,119.00,0.0,714600.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,1.0,1,ไม่ระบุ,ไม่ระบุ,188.50,0.0,451000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,1.0,1,ไม่ระบุ,ไม่ระบุ,207.00,0.0,910000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0


##นักประเมิน

In [603]:
# โหลดไฟล์ Excel
file_path = '/content/คำตอบนักประเมิน(ID).xlsx'
df = pd.read_excel(file_path)
df

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,965000.0,550000.0,"ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,1000000.0,1000000.0,"ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,10873000.0,10000000.0,"ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,1000000.0,1200000.0,ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [604]:
# prompt: เปลี่ยนชื่อ column ID ของ df เป็น id

# Assuming df_house is your DataFrame from the previous code
# Rename the 'ID' column to 'id'
if 'ID' in df.columns:
    df = df.rename(columns={'ID': 'id'})

In [605]:
df

,ID ผู้ประเมิน,post id,id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,965000.0,550000.0,"ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,1000000.0,1000000.0,"ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,10873000.0,10000000.0,"ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,1000000.0,1200000.0,ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [606]:
# prompt: ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID).xlsx" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01

# Filter data for assessor ID 01
df_assessor_01 = df[df['ID ผู้ประเมิน'] == 1]

# Merge the assessor data with the house data
# Assuming 'id' column is common in both dataframes
df_house = pd.merge(df_house, df_assessor_01[['id', 'ประเมินราคาซื้อ-ขาย', 'ราคาทรัพย์']], on='id', how='left')

# Rename columns
df_house = df_house.rename(columns={'ประเมินราคาซื้อ-ขาย': 'estimator', 'ราคาทรัพย์	': 'score'})
df_house


,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,renovate,...,direction,furniture,usable_area,number_of_parking,sell_price,zone,d1000-7/11,d1000-school,estimator,ราคาทรัพย์
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,600000.0,390000.0
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,750000.0,930000.0
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,2800000.0,2994000.0
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0,มะลิวัลย์-ร.8-สนามบิน,0,0,4500000.0,5300000.0
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0,มะลิวัลย์-ร.8-สนามบิน,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,108.76,0.0,437400.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,NaN,NaN
1668,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,119.00,0.0,714600.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,NaN,NaN
1669,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,188.50,0.0,451000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,NaN,NaN
1670,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,207.00,0.0,910000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,NaN,NaN


In [607]:
df_house = df_house.rename(columns={'ราคาทรัพย์	': 'scores'})

In [608]:
# Handle missing values in 'estimator' and 'score' (e.g., fill with 0 or mean)
df_house['estimator'] = df_house['estimator'].fillna(0)  # Or df_house['estimator'].fillna(df_house['estimator'].mean())
df_house['ราคาทรัพย์'] = df_house['ราคาทรัพย์'].fillna(0) # Or df_house['score'].fillna(df_house['score'].mean())
df_house.head(10)

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,renovate,...,direction,furniture,usable_area,number_of_parking,sell_price,zone,d1000-7/11,d1000-school,estimator,ราคาทรัพย์
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่มี,244.0,2.0,390000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,600000.0,390000.0
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่มี,164.0,3.0,820000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,750000.0,930000.0
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,489.0,9.0,3260000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,2800000.0,2994000.0
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,203.0,2.0,5000000.0,มะลิวัลย์-ร.8-สนามบิน,0,0,4500000.0,5300000.0
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,164.0,2.0,3500000.0,มะลิวัลย์-ร.8-สนามบิน,0,0,0.0,0.0
5,20,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านสุรภาธานี,สุรภาธานี,NaN,ขอนแก่น,เมืองขอนแก่น,ศิลา,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,185.0,1.0,2000000.0,ศิลา-โกทา-หนองไผ่-บ้านดอน,0,0,3600000.0,2000000.0
6,23,บ้านเดี่ยว เมืองขอนแก่น,บ้านเดี่ยว เมืองขอนแก่น,NaN,NaN,ขอนแก่น,เมืองขอนแก่น,ศิลา,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,383.0,2.0,5230000.0,ศิลา-โกทา-หนองไผ่-บ้านดอน,0,0,5000000.0,5230000.0
7,25,บ้านเดี่ยว เมืองขอนแก่น โครงการกันยารัตน์ เดอว...,บ้านเดี่ยว เมืองขอนแก่น โครงการกันยารัตน์ เดอว...,กันยารัตน์ เดอะวินเทจ,บริษัท กันยารัตน์ พร็อพเพอร์ตี้ จำกัด,ขอนแก่น,เมืองขอนแก่น,ในเมือง,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,135.0,1.0,2700000.0,โนนทัน-บึงแก่นนคร,0,0,2500000.0,2890000.0
8,28,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านพวงเพชร3,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านพวงเพชร3,พวงเพชร 3,NaN,ขอนแก่น,เมืองขอนแก่น,เมืองเก่า,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,124.0,1.0,1750000.0,กลางเมือง-เมืองเก่า-กุดกว้าง,0,0,1600000.0,1750000.0
9,29,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลพฤกษ์กรีนวิลล์,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลพฤกษ์กรีนวิลล์,ชลพฤกษ์ เลควิลล์,ชลพฤกษ์ กรีนวิลล์,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,ไม่ระบุ,193.0,2.0,4700000.0,ม.ภาค-บ้านกอก-บึงหนองโคตร,0,0,4200000.0,4700000.0


#ตอบคำถาม


##1.zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด

In [609]:
# prompt: zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด เเละเป็นเท่าใด

# Group data by 'zone' and calculate the mean 'sell_price' for each zone
mean_sell_price_by_zone = df_house.groupby('zone')['sell_price'].mean()

# Find the zone with the highest mean 'sell_price'
max_mean_sell_price = mean_sell_price_by_zone.max()
zone_with_highest_price = mean_sell_price_by_zone[mean_sell_price_by_zone == max_mean_sell_price].index[0]

print(f"Zone with the highest average selling price: {zone_with_highest_price}")
print(f"Average selling price in that zone: {max_mean_sell_price}")

Zone with the highest average selling price: ในเมือง-รื่นรมย์-เทพารักษ์
Average selling price in that zone: 7201666.666666667


##2.บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)


In [610]:
# prompt: บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ เเละมี project_name	ว่า

# Find the house with the maximum number of 7-11s within a 1 km radius
max_711_count = df_house['d1000-7/11'].max()
house_with_max_711 = df_house[df_house['d1000-7/11'] == max_711_count]

# Print the price and project name of the house
print(f"The house with the most 7-11s within 1 km radius has a price of: {house_with_max_711['sell_price'].values[0]}")
print(f"Project Name: {house_with_max_711['project_name'].values[0]}")

The house with the most 7-11s within 1 km radius has a price of: 390000.0
Project Name: บ้านวังเวิน


##3.ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [611]:
# prompt: ตำบล (sub_district) ไหน ได้ ราคาทรัพย์ เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

# Group data by 'sub_district' and calculate the mean 'score' for each sub_district
mean_score_by_sub_district = df_house.groupby('sub_district')['ราคาทรัพย์'].mean()

# Find the sub_district with the highest mean 'score'
max_mean_score = mean_score_by_sub_district.max()
sub_district_with_highest_score = mean_score_by_sub_district[mean_score_by_sub_district == max_mean_score].index[0]

print(f"Sub-district with the highest average score: {sub_district_with_highest_score}")
print(f"Average score in that sub-district: {max_mean_score}")

Sub-district with the highest average score: หนองไผ่
Average score in that sub-district: 8960307.692307692
